In [ ]:
from tensorflow.keras import layers
import tensorflow as tf

input_shape = (28, 28, 1)
num_classes = 10

inputs = layers.Input(shape=input_shape)

# Feature Extraction
net = layers.Conv2D(32, 3, padding='SAME')(inputs)
net = layers.Activation('relu')(net)
net = layers.Conv2D(32, 3, padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPool2D((2, 2))(net)
net = layers.Dropout(0.25)(net)

net = layers.Conv2D(64, 3, padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.Conv2D(64, 3, padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPool2D((2, 2))(net)
net = layers.Dropout(0.25)(net)

# Fully Connected
net = layers.Flatten()(net)
net = layers.Dense(512)(net)
net = layers.Activation('relu')(net)
net = layers.Dropout(0.25)(net)
net = layers.Dense(num_classes)(net)
net = layers.Activation('softmax')(net)

model = tf.keras.Model(inputs=inputs, outputs=net, name='Basic_CNN')

loss_func = tf.keras.losses.sparse_categorical_crossentropy
metrics = ['acc']
optm = tf.keras.optimizers.RMSprop()

model.compile(optimizer=optm, loss=loss_func, metrics=metrics)

In [ ]:
from tensorflow.keras import datasets

(X_train, y_train), (X_test, y_test) = datasets.mnist.load_data()

X_train, X_test = X_train / 255.0, X_test / 255.0

X_train = X_train[..., tf.newaxis]
X_test  = X_test[..., tf.newaxis]

X_train.shape, X_test.shape

((60000, 28, 28, 1), (10000, 28, 28, 1))

# EarlyStopping
* 지정한 횟수동안 성능이 개선되지 않으면 학습을 중지

# ModelCheckpoint
* 훈련이 제일 잘된 상태를 파일로 저장(`.h5`)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

es = EarlyStopping(
    monitor="val_loss", # 검증 세트의 loss를 감시하겠다.
    mode='min', # 검증 세트의 loss가 최소가 되는 것을 감시하겠다.
    verbose=1, # 로그 레벨( 신경 x )
    patience=4 # 성능이 나아지길 기다리는 에폭의 횟수, 즉 4회 에폭 훈련동안 성능이 나아지지 않으면 훈련을 종료
)

mc = ModelCheckpoint(
    "best_model.h5", # 체크포인트 파일 저장 이름
    monitor="val_acc", # 저장할 기준이 되는 지표. 검증 세트의 정확도를 검사
    mode="max", # 제일 높은 정확도 일 때 체크포인트를 저장
    verbose=1,
    save_best_only=True # 제일 좋았던 기록만 저장
)

In [ ]:
model.fit(
    X_train,
    y_train,
    validation_split=0.2,
    epochs=100,
    batch_size=32,
    callbacks=[es, mc]
)

Epoch 1/100
1500/1500 [==============================] - 22s 4ms/step - loss: 0.1509 - acc: 0.9529 - val_loss: 0.0434 - val_acc: 0.9872

Epoch 00001: val_acc improved from -inf to 0.98725, saving model to best_model.h5
Epoch 2/100
1500/1500 [==============================] - 6s 4ms/step - loss: 0.0575 - acc: 0.9827 - val_loss: 0.0401 - val_acc: 0.9876

Epoch 00002: val_acc improved from 0.98725 to 0.98758, saving model to best_model.h5
Epoch 3/100
1500/1500 [==============================] - 6s 4ms/step - loss: 0.0490 - acc: 0.9858 - val_loss: 0.0358 - val_acc: 0.9905

Epoch 00003: val_acc improved from 0.98758 to 0.99050, saving model to best_model.h5
Epoch 4/100
1500/1500 [==============================] - 6s 4ms/step - loss: 0.0461 - acc: 0.9869 - val_loss: 0.0428 - val_acc: 0.9902

Epoch 00004: val_acc did not improve from 0.99050
Epoch 5/100
1500/1500 [==============================] - 6s 4ms/step - loss: 0.0447 - acc: 0.9877 - val_loss: 0.0338 - val_acc: 0.9906

Epoch 00005: val_